# Data Extraction

Original Michelin restaurant data was extracted from Kaggle (https://www.kaggle.com/jackywang529/michelin-restaurants).

The data includes one, two, three Michelin star restaurants from the following regions:

Austria, California, Chicago, Croatia, Czech Republic, Denmark, Finland, Greece, Hong Kong, Hungary, Iceland, Macau, Norway, New York City, Poland, Ireland, Rio de Janeiro, Sao Paulo, South Korea, Singapore, Sweden, Taipei, Thailand, Washington DC, and United Kingdom.

The following regions are not included in the dataset:

Belgium, France, Germany, Italy, Japan, Luxembourg, Netherlands, Portugal, China, Spain, and Switzerland.

#### Import Dependencies

In [2]:
# Dependencies
import pandas as pd
from sqlalchemy import create_engine
import requests
import json

# Google developer API key
#from config import g_key

#### Load CSV Files into Pandas Dataframes

In [3]:
# Create file paths
one_star_csv = "Resources/data/one-star-michelin-restaurants.csv"
two_star_csv = "Resources/data/two-stars-michelin-restaurants.csv"
three_star_csv = "Resources/data/three-stars-michelin-restaurants.csv"

In [4]:
# Load one star restaurants
raw_one_star_df = pd.read_csv(one_star_csv)
raw_one_star_df.head()

,name,year,latitude,longitude,city,region,zipCode,cuisine,price,url
0,Kilian Stuba,2019,47.348580,10.17114,Kleinwalsertal,Austria,87568,Creative,$$$$$,https://guide.michelin.com/at/en/vorarlberg/kl...
1,Pfefferschiff,2019,47.837870,13.07917,Hallwang,Austria,5300,Classic cuisine,$$$$$,https://guide.michelin.com/at/en/salzburg-regi...
2,Esszimmer,2019,47.806850,13.03409,Salzburg,Austria,5020,Creative,$$$$$,https://guide.michelin.com/at/en/salzburg-regi...
3,Carpe Diem,2019,47.800010,13.04006,Salzburg,Austria,5020,Market cuisine,$$$$$,https://guide.michelin.com/at/en/salzburg-regi...
4,Edvard,2019,48.216503,16.36852,Wien,Austria,1010,Modern cuisine,$$$$,https://guide.michelin.com/at/en/vienna/wien/r...


In [5]:
# Load two star restaurants
raw_two_star_df = pd.read_csv(two_star_csv)
raw_two_star_df.head()

,name,year,latitude,longitude,city,region,zipCode,cuisine,price,url
0,SENNS.Restaurant,2019,47.83636,13.06389,Salzburg,Austria,5020,Creative,$$$$$,https://guide.michelin.com/at/en/salzburg-regi...
1,Ikarus,2019,47.79536,13.00695,Salzburg,Austria,5020,Creative,$$$$$,https://guide.michelin.com/at/en/salzburg-regi...
2,Mraz & Sohn,2019,48.23129,16.37637,Wien,Austria,1200,Creative,$$$$$,https://guide.michelin.com/at/en/vienna/wien/r...
3,Konstantin Filippou,2019,48.21056,16.37996,Wien,Austria,1010,Modern cuisine,$$$$$,https://guide.michelin.com/at/en/vienna/wien/r...
4,Silvio Nickol Gourmet Restaurant,2019,48.20558,16.37693,Wien,Austria,1010,Modern cuisine,$$$$$,https://guide.michelin.com/at/en/vienna/wien/r...


In [6]:
# Load three star restaurants
raw_three_star_df = pd.read_csv(three_star_csv)
raw_three_star_df.head()

,name,year,latitude,longitude,city,region,zipCode,cuisine,price,url
0,Amador,2019,48.25406,16.35915,Wien,Austria,1190,Creative,$$$$$,https://guide.michelin.com/at/en/vienna/wien/r...
1,Manresa,2019,37.22761,-121.98071,South San Francisco,California,95030,Contemporary,$$$$,https://guide.michelin.com/us/en/california/so...
2,Benu,2019,37.78521,-122.39876,San Francisco,California,94105,Asian,$$$$,https://guide.michelin.com/us/en/california/sa...
3,Quince,2019,37.79762,-122.40337,San Francisco,California,94133,Contemporary,$$$$,https://guide.michelin.com/us/en/california/sa...
4,Atelier Crenn,2019,37.79835,-122.43586,San Francisco,California,94123,Contemporary,$$$$,https://guide.michelin.com/us/en/california/sa...


In [7]:
# Row counts
print("In the raw data there are:")
print(f"{raw_one_star_df.shape[0]} rows for one star restaurants.")
print(f"{raw_two_star_df.shape[0]} rows for two star restaurants.")
print(f"{raw_three_star_df.shape[0]} rows for three star restaurants.")

In the raw data there are:
549 rows for one star restaurants.
110 rows for two star restaurants.
36 rows for three star restaurants.


#### Google API Calls

Our goal is to use the latitude and longitude provided by the original csv data to make API calls to the Google Places API. We will locate the nearest lodging to each Michelin restaurant and extract the name of these hotels to add to the dataset.

In [7]:
# Subset original data frames
one_star_hotel_df = raw_one_star_df[['name', 'latitude', 'longitude']].copy()
two_star_hotel_df = raw_two_star_df[['name', 'latitude', 'longitude']].copy()
three_star_hotel_df = raw_three_star_df[['name', 'latitude', 'longitude']].copy()

In [8]:
# Display copies
one_star_hotel_df.head()

,name,latitude,longitude
0,Kilian Stuba,47.348580,10.17114
1,Pfefferschiff,47.837870,13.07917
2,Esszimmer,47.806850,13.03409
3,Carpe Diem,47.800010,13.04006
4,Edvard,48.216503,16.36852


In [9]:
# Set up url for Google Places API Calls
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
params = {
    "location" : "",
    "rankby" : "distance",
    "type": "lodging",
    "key": g_key,
}

In [10]:
# Run a test API call to check JSON format
lat = 48.25406
lng = 16.35915
params['location'] = f"{lat}, {lng}"
response = requests.get(base_url, params=params).json()
print(json.dumps(response, indent = 4, sort_keys = True))

{
    "html_attributions": [],
    "next_page_token": "ATtYBwK9AwV2nOt1qxi6X0upnfmyX1zyKlLqgx-P2imP1wE-a3juilwJBtTfXhzVyby2w6FVn8hiuZpT573_OxBurRwCUdjmGrBRGbXlgVdZSRiuldtRCgciKMIcBuhlvGCwwjrTcze8arWG_3FhjaDOj9hYzZRI4uOIaaCK0XiQ0XzzMIz99-GIAOYfH7-waM_6NsE1JJta-dkhha0CoPGAoPIDI2I8L9P_ZBxNqZF9OgavCna93WC8y-T0amTVYO8XV3XE41lQoGyMgXXcCS1qah-OqJmVRLMLWmIKJTjLJVlB4XyG5N7SCtNOo41pTuYDhG-qfx-ujSv4NvcMg_lYpEMuK-nUw11pDrUcVz_yzTA6KT4P93MmlXCsCEqZGRJyPiVLBQkO4cUbnZ3JpbLY7YO16qdOiAoyR8tyVqizEo1GQPAMXtm2O6gyNQ",
    "results": [
        {
            "business_status": "OPERATIONAL",
            "geometry": {
                "location": {
                    "lat": 48.25690939999999,
                    "lng": 16.3589786
                },
                "viewport": {
                    "northeast": {
                        "lat": 48.25825088029149,
                        "lng": 16.3602773802915
                    },
                    "southwest": {
                        "lat": 48.255552919

In [11]:
# Loop through rows for coordinates to make API call for one star
for index, row in one_star_hotel_df.iterrows():
    lat = row.latitude
    lng = row.longitude
    params['location'] = f"{lat}, {lng}"
    response = requests.get(base_url, params=params).json()
    one_star_hotel_df.loc[index, "Hotel_Name"] = response["results"][0]["name"]
    print("Hotel Added")

Hotel Added
Hotel Added
Hotel Added
Hotel Added
Hotel Added
Hotel Added
Hotel Added
Hotel Added
Hotel Added
Hotel Added
Hotel Added
Hotel Added
Hotel Added
Hotel Added
Hotel Added
Hotel Added
Hotel Added
Hotel Added
Hotel Added
Hotel Added
Hotel Added
Hotel Added
Hotel Added
Hotel Added
Hotel Added
Hotel Added
Hotel Added
Hotel Added
Hotel Added
Hotel Added
Hotel Added
Hotel Added
Hotel Added
Hotel Added
Hotel Added
Hotel Added
Hotel Added
Hotel Added
Hotel Added
Hotel Added
Hotel Added
Hotel Added
Hotel Added
Hotel Added
Hotel Added
Hotel Added
Hotel Added
Hotel Added
Hotel Added
Hotel Added
Hotel Added
Hotel Added
Hotel Added
Hotel Added
Hotel Added
Hotel Added
Hotel Added
Hotel Added
Hotel Added
Hotel Added
Hotel Added
Hotel Added
Hotel Added
Hotel Added
Hotel Added
Hotel Added
Hotel Added
Hotel Added
Hotel Added
Hotel Added
Hotel Added
Hotel Added
Hotel Added
Hotel Added
Hotel Added
Hotel Added
Hotel Added
Hotel Added
Hotel Added
Hotel Added
Hotel Added
Hotel Added
Hotel Added
Hote

In [12]:
one_star_hotel_df.head()

,name,latitude,longitude,Hotel_Name
0,Kilian Stuba,47.348580,10.17114,Ferienwohnungen Braun
1,Pfefferschiff,47.837870,13.07917,Ferienhaus Essl-Redecsy
2,Esszimmer,47.806850,13.03409,Exerzitienhaus
3,Carpe Diem,47.800010,13.04006,arthotel blaue gans salzburg
4,Edvard,48.216503,16.36852,Palais Hansen Kempinski Vienna


In [13]:
# Loop through rows for coordinates to make API call for two star
for index, row in two_star_hotel_df.iterrows():
    lat = row.latitude
    lng = row.longitude
    params['location'] = f"{lat}, {lng}"
    response = requests.get(base_url, params=params).json()
    two_star_hotel_df.loc[index, "Hotel_Name"] = response["results"][0]["name"]
    print("Hotel Added")

Hotel Added
Hotel Added
Hotel Added
Hotel Added
Hotel Added
Hotel Added
Hotel Added
Hotel Added
Hotel Added
Hotel Added
Hotel Added
Hotel Added
Hotel Added
Hotel Added
Hotel Added
Hotel Added
Hotel Added
Hotel Added
Hotel Added
Hotel Added
Hotel Added
Hotel Added
Hotel Added
Hotel Added
Hotel Added
Hotel Added
Hotel Added
Hotel Added
Hotel Added
Hotel Added
Hotel Added
Hotel Added
Hotel Added
Hotel Added
Hotel Added
Hotel Added
Hotel Added
Hotel Added
Hotel Added
Hotel Added
Hotel Added
Hotel Added
Hotel Added
Hotel Added
Hotel Added
Hotel Added
Hotel Added
Hotel Added
Hotel Added
Hotel Added
Hotel Added
Hotel Added
Hotel Added
Hotel Added
Hotel Added
Hotel Added
Hotel Added
Hotel Added
Hotel Added
Hotel Added
Hotel Added
Hotel Added
Hotel Added
Hotel Added
Hotel Added
Hotel Added
Hotel Added
Hotel Added
Hotel Added
Hotel Added
Hotel Added
Hotel Added
Hotel Added
Hotel Added
Hotel Added
Hotel Added
Hotel Added
Hotel Added
Hotel Added
Hotel Added
Hotel Added
Hotel Added
Hotel Added
Hote

In [14]:
two_star_hotel_df.head()

,name,latitude,longitude,Hotel_Name
0,SENNS.Restaurant,47.83636,13.06389,Inovum
1,Ikarus,47.79536,13.00695,Kärntner Chalets
2,Mraz & Sohn,48.23129,16.37637,Wohnen im Kapellenhof
3,Konstantin Filippou,48.21056,16.37996,PuzzleHotel Apartments Postgasse
4,Silvio Nickol Gourmet Restaurant,48.20558,16.37693,Palais Coburg


In [15]:
# Loop through rows for coordinates to make API call for three star
for index, row in three_star_hotel_df.iterrows():
    lat = row.latitude
    lng = row.longitude
    params['location'] = f"{lat}, {lng}"
    response = requests.get(base_url, params=params).json()
    three_star_hotel_df.loc[index, "Hotel_Name"] = response["results"][0]["name"]
    print("Hotel Added")

Hotel Added
Hotel Added
Hotel Added
Hotel Added
Hotel Added
Hotel Added
Hotel Added
Hotel Added
Hotel Added
Hotel Added
Hotel Added
Hotel Added
Hotel Added
Hotel Added
Hotel Added
Hotel Added
Hotel Added
Hotel Added
Hotel Added
Hotel Added
Hotel Added
Hotel Added
Hotel Added
Hotel Added
Hotel Added
Hotel Added
Hotel Added
Hotel Added
Hotel Added
Hotel Added
Hotel Added
Hotel Added
Hotel Added
Hotel Added
Hotel Added
Hotel Added


In [16]:
three_star_hotel_df.head()

,name,latitude,longitude,Hotel_Name
0,Amador,48.25406,16.35915,Zum Alten Stadttor
1,Manresa,37.22761,-121.98071,Inspired Leadership Group
2,Benu,37.78521,-122.39876,W San Francisco
3,Quince,37.79762,-122.40337,SnapTravel
4,Atelier Crenn,37.79835,-122.43586,The gula place


In [18]:
# Export dataframes as csv files
one_star_hotel_df.to_csv("Resources/data/one_star_hotel.csv", index = False)
two_star_hotel_df.to_csv("Resources/data/two_star_hotel.csv",  index = False)
three_star_hotel_df.to_csv("Resources/data/three_star_hotel.csv",  index = False)

#### Web Scraping

In [11]:
from splinter import Browser
from bs4 import BeautifulSoup as bs
from webdriver_manager.chrome import ChromeDriverManager
import time

In [12]:
def michelin_web_scrape(specials_df):
    
    # Set up splinter if this is the first call of function
    executable_path = {'executable_path': ChromeDriverManager().install()}
    browser = Browser('chrome', **executable_path, headless=False)
    
    # Loop through 
    for index, row in specials_df.iterrows():

        # Visit url and scrape specialties data
        url = row.url
        browser.visit(url)
        time.sleep(1)
        html = browser.html
        soup = bs(html, 'html.parser')
        try:
            scrape_data = soup.find('ul', class_ ="restaurant-details__text-componets--list")
            specialties = scrape_data.find_all('li')


            # Loop through scrape results and append text results to a specials list
            foods = []
            for special in specialties:
                foods.append(special.text)
            print(foods)
            # Add specials list for restaurant to special dataframe
            specials_df.at[index, 'specialties'] = foods
            print('Above specials added successfully.')
        except:
            print("Information not found")
            specials_df.at[index, 'specialties'] = None
            

    # Close browser
    browser.quit()
    print("Scraping complete.")
       
        

##### One Star Scrape

In [20]:
# Create new dataframe with restaurant name and url info to hold scraped data
one_star_specialties_df = raw_one_star_df[['name','url']].copy()
one_star_specialties_df["specialties"] = ""
one_star_specialties_df.head()

,name,url,specialties
0,Kilian Stuba,https://guide.michelin.com/at/en/vorarlberg/kl...,
1,Pfefferschiff,https://guide.michelin.com/at/en/salzburg-regi...,
2,Esszimmer,https://guide.michelin.com/at/en/salzburg-regi...,
3,Carpe Diem,https://guide.michelin.com/at/en/salzburg-regi...,
4,Edvard,https://guide.michelin.com/at/en/vienna/wien/r...,


In [22]:
# Use the scrape function we created to get specialties for each restaurant if applicable
michelin_web_scrape(one_star_specialties_df)



====== WebDriver manager ======
Current google-chrome version is 91.0.4472
Get LATEST driver version for 91.0.4472
Driver [C:\Users\skim3\.wdm\drivers\chromedriver\win32\91.0.4472.101\chromedriver.exe] found in cache


['Steinbutt in schäumender Butter gebraten, Kaviar, Erdäpfelschaum, Sauerrahm', 'Wachtelbrust mit Petersilie souffliert, Steinpilz-Gnocchi, Zwetschge, Madeiraglace', 'Marille und Mandelmilch, weiße Schokolade, Gebirgsenzian, Mohnsablé, Pistazienöl']
Above specials added successfully.
['Saibling, Pfirsich, Verbene, Buttermilch', 'Rehrücken, Zwetschke, Topinambur, Kaffee', 'Walnuss, Savarin und Eis, Traube, Akazie, Pastinake']
Above specials added successfully.
['Jakobsmuschel und Kaisergranat, Tomate, Gurke', 'Kaninchenrücken und Keule mit Limette, Paprika', 'Gefüllte Schokoblätter mit Mango und Pistazien']
Above specials added successfully.
Information not found
['Alpenkaviar, Pinzgauer Rind, Gillardeau Auster', 'Seezunge, Chioggia Beete, Amalfi', 'Kürbis, Opalys, Kurkuma']
Above specials added successfully.
Information not found
['Jakobsmuschel, Sellerie, grüner Apfel, geräucherte Gänseleber', 'Rehrücken, Rote Rüben, fermentiertes Rotkraut', 'Dunkle Schokolade, Erdnuss']
Above special

['Risotto with summer vegetables and Dungeness crab', 'Juniper crusted bison with king trumpet mushrooms and cipollini mushrooms', 'Strawberry Souffle']
Above specials added successfully.
['Oysters, recipe from Guy Savoy', 'Salmon and potato,an iconic dish from the Troigrois family Chef Pierre Troigrois', 'Apple tart, a creation of pastry chef and business partner, Juan Contreras']
Above specials added successfully.
['Oval-shaped ravioli or pancione filled with house made ricotta cheese', 'A roulade of guinea hen breast and sausage made of dark leg meat, duck meat, and pork is tightly rolled with the hen’s skin and roasted', 'Crème fraîche cheesecake']
Above specials added successfully.
['Appetizer assortment', 'Nigiri', 'Their iteration of tiramisu']
Above specials added successfully.
['New Caledonia shrimp with cucumber and watermelon juice', 'Sonoma duck breast with stone fruit and butterbean puree', 'Caramel pot de creme with almond streusel and "river rock"']
Above specials added 

Information not found
['Ceviche seco', 'Wagyu short rib in ancho broth', 'Rajas con crema']
Above specials added successfully.
['Tortellini in brodo', 'Porchetta with salsa verde and asparagus tips', 'Rhurbarb cheesecake with strawberries']
Above specials added successfully.
['Omega 3- three preparations of Northern Adriatic fish', 'Suckling pig, lentils, yellow cabbage, pork rind', 'Apple & Rose- rose ice cream, apple mousse, candied rose petals']
Above specials added successfully.
['Boškarin (indigenous breed of Istrian cattle), local type of pasta, beef tail, truffles, pear', 'Lamb back straps, polenta, eggplant, pine nuts, garlic', 'Baked apple terrine, hazelnut praline paste, vanilla bourbon cream, candied biscuit, chocolate with spices']
Above specials added successfully.
['Zagorski Štrukli with Istrian truffles', 'Wild fish, young beans, yuzu, beurre blanc', 'Apple, rosemary, passion fruit']
Above specials added successfully.
['Carrot mussels and sausage', 'Chicken tingul', 'Fig

Information not found
['Scallops with dashi', '"Potato acting like pasta"', 'Meadowsweet ice cream, ants and caramel']
Above specials added successfully.
['Scallop from Hitra, radishes and clementine', 'Langoustine from Trondheim fjord', 'Whey sorbet with apples from Klakken and rhubarb root oil']
Above specials added successfully.
['Squid, seaweed butter, salted lemon and black garlic', 'Dry-aged quail from Tau, young beetroot and salted cherries', 'Toasted barley ice cream, almond and candied pine cone']
Above specials added successfully.
['Oyster with keta', 'Char nigiri', 'Melon with yuzu']
Above specials added successfully.
Information not found
['Scallop with unripe plums, elderflower and saffron sauce', 'Organic duck with rose hip purée and ragu', 'Honey from Gamle Aker with milk and tagetes']
Above specials added successfully.
['Scallop, barley, beetroot and dill', 'Veal, mushroom, Jerusalem artichoke, carrot shallot sauce', 'Raspberry, mascarpone, hazelnut']
Above specials add

['Mussels with caviar and dill', 'Lamb with roast aubergine and preserved ramson seed caper sauce', 'Sheep’s yoghurt, clove sorbet and damson purée']
Above specials added successfully.
Information not found
['Pistachio tofu with beetroot miso and apple blossom', 'Thornhill duck, cherry-smoked hay leek, scallion, fig and dried rice', 'Strawberry kakigori with koji milk']
Above specials added successfully.
['Lasagne of scallop and langoustine with champagne, sorrel and razor clams', 'Turbot with coco de Paimpol, mussels, crab and fermented horseradish cream', 'Seaweed caramel tart with soda bread crunch and preserved un-waxed salted lemon ice cream']
Above specials added successfully.
['Hand-dived scallop with horseradish, caviar and cucumber', "Grouse 'en crepinette', celeriac, blackberry and sauce grand veneur", 'Chocolate, praline, vinegar and tonka bean']
Above specials added successfully.
Information not found
["Terrine of foie gras and pig's trotter with crabapple jelly and caramel

['Kohlrabi with chervil pesto and ramp cream', 'Venison', 'Chocolate palet']
Above specials added successfully.
Information not found
['Broccoli pierogi, smoked cipollini, goat gouda,potato, and onion soubise', 'Surf and turf with lobster, squab and sauce Américaine', 'Foie gras cake pop with hazelnut, chocolate, amaretto and gold']
Above specials added successfully.
['Braised salsify topped with green apple rectangles and tarragon leaves dotted with fried oysters with a whey sauce', 'Chicken and Dumplings', 'Benne seed ice cream sundae']
Above specials added successfully.
['Bigeye tuna and potato nigiri', 'Grilled ribeye of Japanese Kuroge beef', 'Mango-saffron bavarois']
Above specials added successfully.
['Softshell crab Louie with egg dressing and spiced tomato broth', 'Roast chicken with lemon-garlic panade', 'Warm chocolate souffle cake with buckwheat-miso ice cream']
Above specials added successfully.
['Porcini flan studded with foie gras and porcini cappuccino', 'Medallions of 

Information not found
Information not found
['Crispy duck egg with white beans, chorizo, tomato butter and parsley', 'Loin of Cornish lamb with grilled onion, alliums, lamb jus, capers and herbs', 'Dark chocolate soufflé, chocolate sauce and Chantilly cream']
Above specials added successfully.
['Roast quail with salted caramel shallot tarte Tatin and hazelnuts', 'Fillet of halibut with potato crust, bouillabaisse style sauce and anchovy aioli', 'Trio of raspberry desserts']
Above specials added successfully.
['Carrot cooked in chicken fat with pickled carrot and chicken skin', 'Wagyu beef rump with black garlic emulsion, artichoke and pickled shimeji mushrooms', 'Chocolate and parsnip custard with miso ice cream and passion fruit']
Above specials added successfully.
['Cannelloni of lobster and salted lemon with swede purée, pear, lobster juices', 'Loin and braised breast of Shurdington lamb with Jerusalem artichoke', 'Bitter chocolate and pistachio délice with pistachio ice cream']
Abo

['Roast Orkney scallop with chestnut, bacon and celeriac', 'Herdwick lamb with Jerusalem artichoke and preserved Melfort Farm beetroot', 'Gâteau Opéra, peanut butter and plum']
Above specials added successfully.
['Tortilla Gallega', 'Wood oven-roasted suckling pig', 'Cuajada de turrón, oloroso cream']
Above specials added successfully.
['Spelt and beer with onion and smoked cheddar', 'Duck with cobnuts, damson and elderflower', 'Blackberry, liquorice and wood sorrel']
Above specials added successfully.
Information not found
['Crispy organic egg with kadaif strands and wild mushrooms', 'Lobster ravioli with shellfish velouté', 'Millefeuille with pecan nuts, milk chocolate and salted caramel ice cream']
Above specials added successfully.
['Duck egg burji with lobster', 'Wild muntjac biryani', 'Saffron falooda']
Above specials added successfully.
['Black pepper chicken tikka', 'Chilli and lime lamb chops', 'Chocolate rasmalai']
Above specials added successfully.
['Isle of Skye scallop wit

In [23]:
one_star_specialties_df.head()

,name,url,specialties
0,Kilian Stuba,https://guide.michelin.com/at/en/vorarlberg/kl...,"[Steinbutt in schäumender Butter gebraten, Kav..."
1,Pfefferschiff,https://guide.michelin.com/at/en/salzburg-regi...,"[Saibling, Pfirsich, Verbene, Buttermilch, Reh..."
2,Esszimmer,https://guide.michelin.com/at/en/salzburg-regi...,"[Jakobsmuschel und Kaisergranat, Tomate, Gurke..."
3,Carpe Diem,https://guide.michelin.com/at/en/salzburg-regi...,None
4,Edvard,https://guide.michelin.com/at/en/vienna/wien/r...,"[Alpenkaviar, Pinzgauer Rind, Gillardeau Auste..."


In [25]:
# Exporting dataframe to csv
one_star_specialties_df.to_csv('Resources/data/one_star_specials.csv', index = False)

##### Two Star Scape

In [16]:
# Create new dataframe with restaurant name and url info to hold scraped data
two_star_specialties_df = raw_two_star_df[['name','url']].copy()
two_star_specialties_df["specialties"] = ""
two_star_specialties_df.head()

,name,url,specialties
0,SENNS.Restaurant,https://guide.michelin.com/at/en/salzburg-regi...,
1,Ikarus,https://guide.michelin.com/at/en/salzburg-regi...,
2,Mraz & Sohn,https://guide.michelin.com/at/en/vienna/wien/r...,
3,Konstantin Filippou,https://guide.michelin.com/at/en/vienna/wien/r...,
4,Silvio Nickol Gourmet Restaurant,https://guide.michelin.com/at/en/vienna/wien/r...,


In [17]:
# Use the scrape function we created to get specialties for each restaurant if applicable
michelin_web_scrape(two_star_specialties_df)



====== WebDriver manager ======
Current google-chrome version is 91.0.4472
Get LATEST driver version for 91.0.4472
Driver [C:\Users\skim3\.wdm\drivers\chromedriver\win32\91.0.4472.101\chromedriver.exe] found in cache


['Jakobsmuschel, Tomate, Yuzu, Olivenöl', 'Rehrücken, Gelbe Rübe, Sanddorn, Spitzkraut', 'Topinambur, Mohn, Brombeere']
Above specials added successfully.
['Muscheln, Tomate, Artischocke', 'Steinbarsch, Zwiebel, Speck, Oregano', 'Eingelegte Erdbeeren, Milcheis, Feigenblatt-Öl']
Above specials added successfully.
['Jakobsmuschel mit Marinade', 'Gedämpfter Chinakohl', 'Karotte und Joghurt']
Above specials added successfully.
['Lachsforelle, Shiso, Safran, Kohlrabi', 'Kroatischer Langostino, Kalbszunge, Cochayuyo, Zitrus', 'Kirsche, Apfel, Ingwer, Schokolade']
Above specials added successfully.
['Entenleber, Pilze, Schokolade, Fichtenwipfel', 'Nuart Lamm, Kirsche, Ingwer, Petersilie', 'Quitte, Alpenmilch, Verbene, Schotten']
Above specials added successfully.
['Saibling im Bienenwachs mit gelber Rübe, Pollen und Rahm', 'Waller mit Schönbrunner Calamansi, Mispeln und Kamut', 'Götterfrucht mit Haselnüssen, Herbsttrompeten und Salzkaramell']
Above specials added successfully.
['Artichoke and

['Ostra, torresmo e caipirinha', 'Costela de porco, cerveja preta e baroa', 'Mamão verde e queijadinha']
Above specials added successfully.
Information not found
['Salada de cogumelos yanomami', 'Cupim, mandioquinha e toffee de baunilha do Cerrado', 'Mochi de tapioca e rapadura com sorvete de erva-mate']
Above specials added successfully.
Information not found
Information not found
Information not found
Information not found
Information not found
Information not found
Information not found
Information not found
Information not found
Information not found
Information not found
['Grilled langoustine with "Swedish curry" and herbs', 'Quail with sprouting broccoli and buckwheat', 'Sorrel ice cream with wild blueberries and tarragon']
Above specials added successfully.
['Crab tartlet with trout roe & herb salad', 'Pike perch with seaweed & sauce of 200 year old Mahogany clam', 'Rhubarb with tagetes from our garden & chard cooked in ättika']
Above specials added successfully.
['Cabbage with 

In [18]:
two_star_specialties_df.head()

,name,url,specialties
0,SENNS.Restaurant,https://guide.michelin.com/at/en/salzburg-regi...,"[Jakobsmuschel, Tomate, Yuzu, Olivenöl, Rehrüc..."
1,Ikarus,https://guide.michelin.com/at/en/salzburg-regi...,"[Muscheln, Tomate, Artischocke, Steinbarsch, Z..."
2,Mraz & Sohn,https://guide.michelin.com/at/en/vienna/wien/r...,"[Jakobsmuschel mit Marinade, Gedämpfter Chinak..."
3,Konstantin Filippou,https://guide.michelin.com/at/en/vienna/wien/r...,"[Lachsforelle, Shiso, Safran, Kohlrabi, Kroati..."
4,Silvio Nickol Gourmet Restaurant,https://guide.michelin.com/at/en/vienna/wien/r...,"[Entenleber, Pilze, Schokolade, Fichtenwipfel,..."


In [27]:
# Exporting dataframe to csv
two_star_specialties_df.to_csv('Resources/data/two_star_specials.csv', index = False)

##### Three Star Scrape

In [10]:
# Create new dataframe with restaurant name and url info to hold scraped data
three_star_specialties_df = raw_three_star_df[['name','url']].copy()
three_star_specialties_df["specialties"] = ""
three_star_specialties_df.head()

,name,url,specialties
0,Amador,https://guide.michelin.com/at/en/vienna/wien/r...,
1,Manresa,https://guide.michelin.com/us/en/california/so...,
2,Benu,https://guide.michelin.com/us/en/california/sa...,
3,Quince,https://guide.michelin.com/us/en/california/sa...,
4,Atelier Crenn,https://guide.michelin.com/us/en/california/sa...,


In [13]:
# Use the scrape function we created to get specialties for each restaurant if applicable
michelin_web_scrape(three_star_specialties_df)



====== WebDriver manager ======
Current google-chrome version is 91.0.4472
Get LATEST driver version for 91.0.4472
There is no [win32] chromedriver for browser 91.0.4472 in cache
Get LATEST driver version for 91.0.4472
Trying to download new driver from https://chromedriver.storage.googleapis.com/91.0.4472.101/chromedriver_win32.zip
Driver has been saved in cache [C:\Users\skim3\.wdm\drivers\chromedriver\win32\91.0.4472.101]


['Ochsenherztomate, Kaisergranat, Mascarpone, Kalbsbries', 'Taube, Mango, Kokos, Purple Curry', 'Rote Rübe, Himbeere, Tonkabohne']
Above specials added successfully.
['Lightly smoked striped jack, brassicas', 'Sweetbreads, mushroom, toasted levain soup', 'Milk chocolate, kuri squash, toasted milk']
Above specials added successfully.
['Lobster Xiao Long Bao with Black Vinegar and House-Made Soy Sauce', 'Barbecued Wolfe Ranch Quail with XO Sauce and Steamed Bao with Black Truffle Cream', 'Sonoma Creamery Milk Pudding with Salted Caramel']
Above specials added successfully.
['Freshly Dug Potatoes with oysters, kale, and vermouth butter sauce', 'Squab with black truffle, foie gras and red currant', 'Caramel apple with rosemary, celery seed, pine nut and rum raisin']
Above specials added successfully.
['Seed and grains, English peas and mint', 'Striped bass, boudin noir and porcini', 'Honey, sapote and pollen, exotic fruit, coconut and pineapple']
Above specials added successfully.
['Oyster

In [14]:
three_star_specialties_df.head(36)

,name,url,specialties
0,Amador,https://guide.michelin.com/at/en/vienna/wien/r...,"[Ochsenherztomate, Kaisergranat, Mascarpone, K..."
1,Manresa,https://guide.michelin.com/us/en/california/so...,"[Lightly smoked striped jack, brassicas, Sweet..."
2,Benu,https://guide.michelin.com/us/en/california/sa...,[Lobster Xiao Long Bao with Black Vinegar and ...
3,Quince,https://guide.michelin.com/us/en/california/sa...,"[Freshly Dug Potatoes with oysters, kale, and ..."
4,Atelier Crenn,https://guide.michelin.com/us/en/california/sa...,"[Seed and grains, English peas and mint, Strip..."
5,The French Laundry,https://guide.michelin.com/us/en/california/sa...,"[Oysters and pearls, Elysian Fields Farm lamb ..."
6,The Restaurant at Meadowood,https://guide.michelin.com/us/en/california/sa...,None
7,SingleThread,https://guide.michelin.com/us/en/california/sa...,"[Pumpkin tartare with Dungeness crab, Smoked O..."
8,Alinea,https://guide.michelin.com/us/en/illinois/chic...,[Scallop noodle with Thai curry sauce and drie...
9,Geranium,https://guide.michelin.com/dk/en/capital-regio...,[Grilled lobster and milk with fermented carro...


In [28]:
# Exporting dataframe to csv
three_star_specialties_df.to_csv('Resources/data/three_star_specials.csv', index = False)

# Data Transformation

In [8]:
# Load exported csv files back into data frames
one_star_hotel_df = pd.read_csv("Resources/data/one_star_hotel.csv")
two_star_hotel_df = pd.read_csv("Resources/data/two_star_hotel.csv")
three_star_hotel_df = pd.read_csv("Resources/data/three_star_hotel.csv")
one_star_specialties_df = pd.read_csv("Resources/data/one_star_specials.csv")
two_star_specialties_df = pd.read_csv("Resources/data/two_star_specials.csv")
three_star_specialties_df = pd.read_csv("Resources/data/three_star_specials.csv")

In [9]:
type(raw_one_star_df.latitude[0])

numpy.float64

In [10]:
type(one_star_hotel_df.latitude[0])

numpy.float64

In [18]:
# Join the original, hotel, and specialty dataframes for each level of michelin star 
# One star
full_one_star_michelin_df = raw_one_star_df.merge(one_star_hotel_df)
full_one_star_michelin_df = final_one_star_michelin_df.merge(one_star_specialties_df)
full_one_star_michelin_df.head()

,name,year,latitude,longitude,city,region,zipCode,cuisine,price,url,Hotel_Name,specialties
0,Kilian Stuba,2019,47.348580,10.17114,Kleinwalsertal,Austria,87568,Creative,$$$$$,https://guide.michelin.com/at/en/vorarlberg/kl...,Ferienwohnungen Braun,"['Steinbutt in schäumender Butter gebraten, Ka..."
1,Pfefferschiff,2019,47.837870,13.07917,Hallwang,Austria,5300,Classic cuisine,$$$$$,https://guide.michelin.com/at/en/salzburg-regi...,Ferienhaus Essl-Redecsy,"['Saibling, Pfirsich, Verbene, Buttermilch', '..."
2,Carpe Diem,2019,47.800010,13.04006,Salzburg,Austria,5020,Market cuisine,$$$$$,https://guide.michelin.com/at/en/salzburg-regi...,arthotel blaue gans salzburg,NaN
3,Edvard,2019,48.216503,16.36852,Wien,Austria,1010,Modern cuisine,$$$$,https://guide.michelin.com/at/en/vienna/wien/r...,Palais Hansen Kempinski Vienna,"['Alpenkaviar, Pinzgauer Rind, Gillardeau Aust..."
4,Pramerl & the Wolf,2019,48.209450,16.37174,Wien,Austria,1090,Creative,$$$$$,https://guide.michelin.com/at/en/vienna/wien/r...,Kleine Weihnachtsgeschenke,"['Jakobsmuschel, Sellerie, grüner Apfel, geräu..."


In [20]:
# Two star
full_two_star_michelin_df = raw_two_star_df.merge(two_star_hotel_df)
full_two_star_michelin_df = full_two_star_michelin_df.merge(two_star_specialties_df)
full_two_star_michelin_df.head()

,name,year,latitude,longitude,city,region,zipCode,cuisine,price,url,Hotel_Name,specialties
0,Mraz & Sohn,2019,48.23129,16.37637,Wien,Austria,1200,Creative,$$$$$,https://guide.michelin.com/at/en/vienna/wien/r...,Wohnen im Kapellenhof,"['Jakobsmuschel mit Marinade', 'Gedämpfter Chi..."
1,Konstantin Filippou,2019,48.21056,16.37996,Wien,Austria,1010,Modern cuisine,$$$$$,https://guide.michelin.com/at/en/vienna/wien/r...,PuzzleHotel Apartments Postgasse,"['Lachsforelle, Shiso, Safran, Kohlrabi', 'Kro..."
2,Silvio Nickol Gourmet Restaurant,2019,48.20558,16.37693,Wien,Austria,1010,Modern cuisine,$$$$$,https://guide.michelin.com/at/en/vienna/wien/r...,Palais Coburg,"['Entenleber, Pilze, Schokolade, Fichtenwipfel..."
3,Baumé,2019,37.42850,-122.14272,South San Francisco,California,94306,Contemporary,$$$$,https://guide.michelin.com/us/en/california/so...,Coronet Motel,"['Artichoke and truffle soup', 'Le Jaune d’oeu..."
4,Commis,2019,37.82476,-122.25505,San Francisco,California,94601,Contemporary,$$$$,https://guide.michelin.com/us/en/california/sa...,SeniorCareList,"['Carrot tartare with young coconut and mint',..."


In [21]:
# Three star
full_three_star_michelin_df = raw_three_star_df.merge(three_star_hotel_df)
full_three_star_michelin_df = full_three_star_michelin_df.merge(three_star_specialties_df)
full_three_star_michelin_df.head()

,name,year,latitude,longitude,city,region,zipCode,cuisine,price,url,Hotel_Name,specialties
0,Manresa,2019,37.22761,-121.98071,South San Francisco,California,95030,Contemporary,$$$$,https://guide.michelin.com/us/en/california/so...,Inspired Leadership Group,"['Lightly smoked striped jack, brassicas', 'Sw..."
1,Quince,2019,37.79762,-122.40337,San Francisco,California,94133,Contemporary,$$$$,https://guide.michelin.com/us/en/california/sa...,SnapTravel,"['Freshly Dug Potatoes with oysters, kale, and..."
2,Atelier Crenn,2019,37.79835,-122.43586,San Francisco,California,94123,Contemporary,$$$$,https://guide.michelin.com/us/en/california/sa...,The gula place,"['Seed and grains, English peas and mint', 'St..."
3,The French Laundry,2019,38.40443,-122.36474,San Francisco,California,94599,Contemporary,$$$$,https://guide.michelin.com/us/en/california/sa...,"Lavender, A Four Sisters Inn","['Oysters and pearls', 'Elysian Fields Farm la..."
4,The Restaurant at Meadowood,2019,38.52025,-122.46479,San Francisco,California,94574,Contemporary,$$$$,https://guide.michelin.com/us/en/california/sa...,Meadowood Napa Valley,NaN


In [67]:
# Keep columns ["name", "city", "region", "cuisine", "url", "Hotel_Name", "specialties"]

final_one_star_michelin_df = full_one_star_michelin_df[["name", "city", "region", "cuisine", "url", "Hotel_Name", "specialties"]].copy()
final_two_star_michelin_df = full_two_star_michelin_df[["name", "city", "region", "cuisine", "url", "Hotel_Name", "specialties"]].copy()
final_three_star_michelin_df = full_three_star_michelin_df[["name", "city", "region", "cuisine", "url", "Hotel_Name", "specialties"]].copy()

In [68]:
# Function to change specialties column values from strings back to lists
def string_to_list(specials):
    specials_list = [special.replace("'", "") for special in specials.replace("[", "").replace("]", "").split("', ")]
    return specials_list

In [69]:
# Use function on dataframes

# One star
for index, row in final_one_star_michelin_df.iterrows():
    # Ignore N/A values when using function
    if not(isinstance(row.specialties, float)):
        row.specialties = string_to_list(row.specialties)
        

# Two star
for index, row in final_two_star_michelin_df.iterrows():
    # Ignore N/A values when using function
    if not(isinstance(row.specialties, float)):
        row.specialties = string_to_list(row.specialties)

# Three star
for index, row in final_three_star_michelin_df.iterrows():
    # Ignore N/A values when using function
    if not(isinstance(row.specialties, float)):
        row.specialties = string_to_list(row.specialties)

In [73]:
# Check that specialties are lists now and display dataframe

In [70]:
print(type(final_one_star_michelin_df.specialties[0]))
final_one_star_michelin_df.head()

<class 'list'>


,name,city,region,cuisine,url,Hotel_Name,specialties
0,Kilian Stuba,Kleinwalsertal,Austria,Creative,https://guide.michelin.com/at/en/vorarlberg/kl...,Ferienwohnungen Braun,"[Steinbutt in schäumender Butter gebraten, Kav..."
1,Pfefferschiff,Hallwang,Austria,Classic cuisine,https://guide.michelin.com/at/en/salzburg-regi...,Ferienhaus Essl-Redecsy,"[Saibling, Pfirsich, Verbene, Buttermilch, Reh..."
2,Carpe Diem,Salzburg,Austria,Market cuisine,https://guide.michelin.com/at/en/salzburg-regi...,arthotel blaue gans salzburg,NaN
3,Edvard,Wien,Austria,Modern cuisine,https://guide.michelin.com/at/en/vienna/wien/r...,Palais Hansen Kempinski Vienna,"[Alpenkaviar, Pinzgauer Rind, Gillardeau Auste..."
4,Pramerl & the Wolf,Wien,Austria,Creative,https://guide.michelin.com/at/en/vienna/wien/r...,Kleine Weihnachtsgeschenke,"[Jakobsmuschel, Sellerie, grüner Apfel, geräuc..."


In [71]:
print(type(final_two_star_michelin_df.specialties[0]))
final_two_star_michelin_df.head()

<class 'list'>


,name,city,region,cuisine,url,Hotel_Name,specialties
0,Mraz & Sohn,Wien,Austria,Creative,https://guide.michelin.com/at/en/vienna/wien/r...,Wohnen im Kapellenhof,"[Jakobsmuschel mit Marinade, Gedämpfter Chinak..."
1,Konstantin Filippou,Wien,Austria,Modern cuisine,https://guide.michelin.com/at/en/vienna/wien/r...,PuzzleHotel Apartments Postgasse,"[Lachsforelle, Shiso, Safran, Kohlrabi, Kroati..."
2,Silvio Nickol Gourmet Restaurant,Wien,Austria,Modern cuisine,https://guide.michelin.com/at/en/vienna/wien/r...,Palais Coburg,"[Entenleber, Pilze, Schokolade, Fichtenwipfel,..."
3,Baumé,South San Francisco,California,Contemporary,https://guide.michelin.com/us/en/california/so...,Coronet Motel,"[Artichoke and truffle soup, Le Jaune d’oeuf f..."
4,Commis,San Francisco,California,Contemporary,https://guide.michelin.com/us/en/california/sa...,SeniorCareList,"[Carrot tartare with young coconut and mint, S..."


In [72]:
print(type(final_three_star_michelin_df.specialties[0]))
final_three_star_michelin_df.head()

<class 'list'>


,name,city,region,cuisine,url,Hotel_Name,specialties
0,Manresa,South San Francisco,California,Contemporary,https://guide.michelin.com/us/en/california/so...,Inspired Leadership Group,"[Lightly smoked striped jack, brassicas, Sweet..."
1,Quince,San Francisco,California,Contemporary,https://guide.michelin.com/us/en/california/sa...,SnapTravel,"[Freshly Dug Potatoes with oysters, kale, and ..."
2,Atelier Crenn,San Francisco,California,Contemporary,https://guide.michelin.com/us/en/california/sa...,The gula place,"[Seed and grains, English peas and mint, Strip..."
3,The French Laundry,San Francisco,California,Contemporary,https://guide.michelin.com/us/en/california/sa...,"Lavender, A Four Sisters Inn","[Oysters and pearls, Elysian Fields Farm lamb ..."
4,The Restaurant at Meadowood,San Francisco,California,Contemporary,https://guide.michelin.com/us/en/california/sa...,Meadowood Napa Valley,NaN


# Data Loading

In [28]:
import pymongo

In [29]:
# Create a connection to Mongo
conn = 'mongodb://localhost:27017'
client = pymongo.MongoClient(conn)

In [74]:
# Create new database for michelin data
db = client.michelinDB

In [75]:
# Create collection and insert documents for each level of restaurant

db.one_star.insert_many(final_one_star_michelin_df.to_dict('records'))
db.two_star.insert_many(final_two_star_michelin_df.to_dict('records'))
db.three_star.insert_many(final_three_star_michelin_df.to_dict('records'))

##### Confirm that dataframes were succesfully uploaded to MongoDB

In [86]:
# Display first five restaurants with one star
one_star = db.one_star.find()
counter = 0
print("These are the first 5 documents in the one_star collection:\n")
for restaurant in one_star:
    if counter >= 5:
        break
    print(restaurant)
    counter+=1

These are the first 5 documents in the one_star collection:

{'_id': ObjectId('60c5065b6d98e129005c0f1f'), 'name': 'Kilian Stuba', 'city': 'Kleinwalsertal', 'region': 'Austria', 'cuisine': 'Creative', 'url': 'https://guide.michelin.com/at/en/vorarlberg/kleinwalsertal/restaurant/kilian-stuba', 'Hotel_Name': 'Ferienwohnungen Braun', 'specialties': ['Steinbutt in schäumender Butter gebraten, Kaviar, Erdäpfelschaum, Sauerrahm', 'Wachtelbrust mit Petersilie souffliert, Steinpilz-Gnocchi, Zwetschge, Madeiraglace', 'Marille und Mandelmilch, weiße Schokolade, Gebirgsenzian, Mohnsablé, Pistazienöl']}
{'_id': ObjectId('60c5065b6d98e129005c0f20'), 'name': 'Pfefferschiff', 'city': 'Hallwang', 'region': 'Austria', 'cuisine': 'Classic cuisine', 'url': 'https://guide.michelin.com/at/en/salzburg-region/hallwang/restaurant/pfefferschiff', 'Hotel_Name': 'Ferienhaus Essl-Redecsy', 'specialties': ['Saibling, Pfirsich, Verbene, Buttermilch', 'Rehrücken, Zwetschke, Topinambur, Kaffee', 'Walnuss, Savarin und

In [87]:
# Display first five restaurants with two stars
one_star = db.two_star.find()
counter = 0
print("These are the first 5 documents in the two_star collection:\n")
for restaurant in one_star:
    if counter >= 5:
        break
    print(restaurant)
    counter+=1

These are the first 5 documents in the two_star collection:

{'_id': ObjectId('60c5065b6d98e129005c1072'), 'name': 'Mraz & Sohn', 'city': 'Wien', 'region': 'Austria', 'cuisine': 'Creative', 'url': 'https://guide.michelin.com/at/en/vienna/wien/restaurant/mraz-sohn', 'Hotel_Name': 'Wohnen im Kapellenhof', 'specialties': ['Jakobsmuschel mit Marinade', 'Gedämpfter Chinakohl', 'Karotte und Joghurt']}
{'_id': ObjectId('60c5065b6d98e129005c1073'), 'name': 'Konstantin Filippou', 'city': 'Wien', 'region': 'Austria', 'cuisine': 'Modern cuisine', 'url': 'https://guide.michelin.com/at/en/vienna/wien/restaurant/konstantin-filippou', 'Hotel_Name': 'PuzzleHotel Apartments Postgasse', 'specialties': ['Lachsforelle, Shiso, Safran, Kohlrabi', 'Kroatischer Langostino, Kalbszunge, Cochayuyo, Zitrus', 'Kirsche, Apfel, Ingwer, Schokolade']}
{'_id': ObjectId('60c5065b6d98e129005c1074'), 'name': 'Silvio Nickol Gourmet Restaurant', 'city': 'Wien', 'region': 'Austria', 'cuisine': 'Modern cuisine', 'url': 'https

In [85]:
# Display first five restaurants with three stars
one_star = db.three_star.find()
counter = 0
print("These are the first 5 documents in the three_star collection:\n")
for restaurant in one_star:
    if counter >= 5:
        break
    print(restaurant)
    counter+=1

These are the first 5 documents in the three_star collection

{'_id': ObjectId('60c5065b6d98e129005c10b3'), 'name': 'Manresa', 'city': 'South San Francisco', 'region': 'California', 'cuisine': 'Contemporary', 'url': 'https://guide.michelin.com/us/en/california/south-san-francisco/restaurant/manresa', 'Hotel_Name': 'Inspired Leadership Group', 'specialties': ['Lightly smoked striped jack, brassicas', 'Sweetbreads, mushroom, toasted levain soup', 'Milk chocolate, kuri squash, toasted milk']}
{'_id': ObjectId('60c5065b6d98e129005c10b4'), 'name': 'Quince', 'city': 'San Francisco', 'region': 'California', 'cuisine': 'Contemporary', 'url': 'https://guide.michelin.com/us/en/california/san-francisco/restaurant/quince', 'Hotel_Name': 'SnapTravel', 'specialties': ['Freshly Dug Potatoes with oysters, kale, and vermouth butter sauce', 'Squab with black truffle, foie gras and red currant', 'Caramel apple with rosemary, celery seed, pine nut and rum raisin']}
{'_id': ObjectId('60c5065b6d98e129005c10